In [1]:
from utils.ML2.utils import *
import warnings
warnings.filterwarnings('ignore')

# Payload Estimation

In robotics, the identification of mass, is crucial for ensuring accurate and safe operation, particularly in tasks that involve dynamic interactions with the environment like pick and place. Traditional methods of measuring weight often require direct physical measurements, which can be both time-consuming and impractical in automated systems where multiple types of payloads are involved. This application of a the developed model offers a significant advantage by enabling the indirect estimation of these parameters through the analysis of predicted forces. This increases the adaptability of the robot to various tools and payloads without the need for frequent recalibration or manual measurements.

Another particularly valuable use case for this technology is in detecting situations where a payload, after being picked up, might slip down . Such a slip can be quickly detected by the virtual sensor. Without this detection capability, the robot could experience unexpected torques and forces, leading to a sudden jerk or unplanned movement, which could compromise the precision of the task or even damage the robot and the payload. By identifying these changes in real-time, the system can adjust its operations to compensate, ensuring smooth and safe handling of the payload.


# Implementation

The estimation of mass using a virtual joint torque sensor is based on the analysis of the forces acting on the tool as it interacts with the environment. The forces predicted by the sensor, in the $X$, $Y$, and $Z$ directions, can be used to compute the overall magnitude of the force vector, which, when divided by the gravitational constant, provides an estimate of the tool's mass.

In a static or quasi-static condition, the forces measured by the virtual joint torque sensor are primarily due to the weight of the tool and any attached payload. The relationship between the force components and the mass of the tool can be expressed as:

$$|\vec{F}| = \sqrt{F_x^2 + F_y^2 + F_z^2}$$

where:

- $F_x$, $F_y$, and $F_z$ are the force components in the $X$, $Y$, and $Z$ directions, respectively.
- $|\vec{F}|$ is the magnitude of the resultant force vector.

Given that the force due to gravity is $F = m \cdot g$, where $g$ is the acceleration due to gravity (approximately $9.81 \, \text{m/s}^2$), the mass $m$ can be estimated as:

$$m = \frac{|\vec{F}|}{g}$$

To ensure accuracy, measurements are made over a period of time, and the average of the calculated mass values is taken to provide a more reliable estimate. This approach helps to reduce the impact of sensor noise, and compensate for any variability in the conditions during the measurements, leading to a more accurate estimation of the tool's mass.

This equation assumes that the forces measured are primarily due to gravity, and thus the resultant force vector provides a direct indication of the tool's mass.


To validate the performance of the model in this application, an eccentric mass as shown in Figure 1 was attached to the robot and the robot was moved along a generated trajectory as described in chapter 4 section 1. A velocity scaling of 5% was applied so as to maintain a quasi-static condition. 

![drawing](./utils/ML2/application/eccentrictool.jpeg){ width=50% }

In [4]:
y_max = torch.load('utils/ML2/models/y_max.pt',map_location=torch.device('cpu'))
y_min = torch.load('utils/ML2/models/y_min.pt',map_location=torch.device('cpu'))

dataset = CustomDatasetFromCSV(csv_path = 'utils/ML2/application/lara8_0.5.csv', mode="test")
test_dataloader = DataLoader(dataset, batch_size=100, shuffle=False)

model = torch.jit.load('utils/ML2/models/model.pt', map_location=torch.device('cpu'))
model.eval()

pred_list = []
with torch.no_grad():    
    for i, (X,y) in enumerate(tqdm(test_dataloader)):
        pred = model(X)
        #denormalize predictions
        pred = (pred+1)/2* (y_max - y_min) + y_min
        pred_list.append(pred)
# Denormalize the data
y_truth = ((test_dataloader.dataset.y + 1) / 2 * (y_max - y_min) + y_min).cpu().numpy()
pred_list = torch.cat(pred_list).cpu().numpy()

y_truth = y_truth[116:]

# Example usage of your specific mass calculation

FTx = y_truth[0:100, 0]
FTy = y_truth[0:100, 1]
FTz = y_truth[0:100, 2]

mass_estimate = np.sqrt(Fx**2 + Fy**2 + Fz**2).mean() / 9.81
mass_actual = np.sqrt(FTx**2 + FTy**2 + FTz**2).mean() / 9.81
print("Estimated mass: ", mass_estimate.item())
print("Actual mass: ", mass_actual.item())

100%|██████████| 280/280 [01:31<00:00,  3.05it/s]

Estimated mass:  2.7590842057440987
Actual mass:  2.7074553307893927


The model estimated the mass to be **2.71kg** whereas the mass estimated from FTS measurements was **2.75kg**. The following table shows the model estimation on a number of different payloads.This demonstrates that the model can be used to estimate the mass of a a tool or object attached to the robot's TCP reliably. 

| **Payload** | **Estimated Mass (kg)** | **Actual Mass (kg)** | **Absolute Error (kg)** |
|-------------|--------------------------|----------------------|-------------------------|
| Payload 1   | 2.71                      | 2.75                 | 0.04                    |
| Payload 2   | 1.5554                    | 1.5223               | 0.0331                  |
| Payload 3   | 7.2118                    | 7.1823               | 0.0295                   |

